In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import VBox
from IPython.display import display, HTML
import plotly.io as pio

# Function to calculate and plot the FIRE plan
def calculate_fire(
        current_age,
        target_retirement_age,
        current_salary,
        salary_growth_rate, 
        percent_earnings_invested, 
        annual_returns_pre_retirement, 
        annual_returns_post_retirement, 
        initial_inflation_rate, 
        pre_retirement_expenses):
    
    # Create an empty DataFrame to hold the calculations
    years = list(range(0, 90 - current_age + 1))  # Assuming analysis till age 90
    df = pd.DataFrame({'Year': years})

    # Calculations for Age
    df['Age'] = df['Year'] + current_age
    
    # Earning Toggle: 1 if earning, 0 if retired
    df['Earning Toggle'] = (df['Age'] < target_retirement_age).astype(int)
    
    # Annual Salary Growth Rate
    df['Annual Salary Growth %'] = salary_growth_rate
    df.loc[6::5, 'Annual Salary Growth %'] *= 0.8  # Adjust salary growth rate after certain years
    
    # Annual Salary Calculation
    df['Annual Salary'] = current_salary * df['Earning Toggle'] * (1 + df['Annual Salary Growth %']).cumprod()
    df['Annual Salary'] = df['Annual Salary'].shift().fillna(current_salary * df['Earning Toggle'])  # Adjust for year 0
    
    # Annual Investments and Expenses (pre-retirement)
    df['Annual Investments'] = df['Annual Salary'] * percent_earnings_invested
    df['Annual Expenses (pre-retirement)'] = df['Annual Salary'] - df['Annual Investments']
    
    # Future Value of Investments at Retirement
    df['Future Value of Investments at Retirement'] = df['Annual Investments'] * ((1 + annual_returns_pre_retirement) ** (target_retirement_age - df['Age']))
    
    # Annual Inflation Rate
    df['Annual Inflation %'] = initial_inflation_rate
    df.loc[6::5, 'Annual Inflation %'] *= 0.84  # Adjust inflation rate after certain years
    
    # Future Value of est. Expense
    df['Future Value of est. Expense'] = pre_retirement_expenses * (1 + df['Annual Inflation %']).cumprod()
    df.loc[0, 'Future Value of est. Expense'] = pre_retirement_expenses  # Ensure year 0 has the initial value
    
    # Spending Toggle: 1 if spending (post-retirement), 0 if not
    df['Spending Toggle'] = (df['Age'] >= target_retirement_age).astype(int)
    df['Ad-Hoc Expenses'] = 0  # Can be manually updated later
    
    # Calculate Total Corpus initially from Investments
    df['Total Corpus'] = df['Future Value of Investments at Retirement'].cumsum()
    
    # Calculate Returns on Corpus post-retirement
    df['Returns on Corpus post-retirement'] = df['Spending Toggle'] * df['Total Corpus'].shift() * annual_returns_post_retirement
    df['Returns on Corpus post-retirement'] = df['Returns on Corpus post-retirement'].fillna(0)
    
    # Calculate Required Withdrawals
    df['Required Withdrawals'] = -1 * (df['Spending Toggle'] * df['Future Value of est. Expense'] - df['Ad-Hoc Expenses'])
    
    # Update Total Corpus with returns and withdrawals
    df['Total Corpus'] = df['Total Corpus'] + df['Returns on Corpus post-retirement'].cumsum() + df['Required Withdrawals'].cumsum()
    
    # Convert Total Corpus to crores
    df['Total Corpus (₹ Crs.)'] = df['Total Corpus'] / 10**7
    df['Annual Salary (₹ Crs.)'] = df['Annual Salary'] / 10**7
    df['Annual Investments (₹ Crs.)'] = df['Annual Investments'] / 10**7
    df['Annual Expenses (pre-retirement) (₹ Crs.)'] = df['Annual Expenses (pre-retirement)'] / 10**7
    df['Future Value of Investments at Retirement (₹ Crs.)'] = df['Future Value of Investments at Retirement'] / 10**7
    df['Future Value of est. Expense (₹ Crs.)'] = df['Future Value of est. Expense'] / 10**7
    
    # Separate positive and negative values
    df['Total Corpus Positive'] = df['Total Corpus (₹ Crs.)'].apply(lambda x: x if x >= 0 else None)
    df['Total Corpus Negative'] = df['Total Corpus (₹ Crs.)'].apply(lambda x: x if x < 0 else None)
    
    # Find the age where total corpus becomes zero
    zero_corpus_age = df.loc[df['Total Corpus (₹ Crs.)'] <= 0, 'Age'].min()
    
    # Summary calculations
    cumulative_fv_pre_retirement_investments = df['Future Value of Investments at Retirement'].sum()
    returns_post_retirement_corpus = df['Returns on Corpus post-retirement'].sum()
    total_required_withdrawals = df['Required Withdrawals'].sum()
    shortfall = -1* (cumulative_fv_pre_retirement_investments + returns_post_retirement_corpus + total_required_withdrawals)
    
    # Print summary
    print(f'Cumulative FV of pre-retirement Investments: ₹ {cumulative_fv_pre_retirement_investments / 10**7:.2f} Crs.')
    print(f'Returns on Post-Retirement Corpus: ₹ {returns_post_retirement_corpus / 10**7:.2f} Crs.')
    print(f'Total Required Withdrawals: ₹ {total_required_withdrawals / 10**7:.2f} Crs.')
    print(f'Shortfall: ₹ {shortfall / 10**7:.2f} Crs.')
    print(f'Age when Total Corpus becomes Zero: {zero_corpus_age if not np.isnan(zero_corpus_age) else "Never"}')
    
    # Enhanced Plotly Visualization
    fig = go.Figure()
    
    # Plot positive values in blue
    fig.add_trace(go.Scatter(
        x=df['Age'], 
        y=df['Total Corpus Positive'], 
        mode='lines+markers',
        name='Total Corpus (Positive)',
        line=dict(color='royalblue', width=2),
        marker=dict(size=5)
    ))
    
    # Plot negative values in red
    fig.add_trace(go.Scatter(
        x=df['Age'], 
        y=df['Total Corpus Negative'], 
        mode='lines+markers',
        name='Total Corpus (Negative)',
        line=dict(color='red', width=2),
        marker=dict(size=5)
    ))
    
    # Add vertical line for retirement age
    fig.add_shape(type="line",
                  x0=target_retirement_age, x1=target_retirement_age,
                  y0=df['Total Corpus (₹ Crs.)'].min(), y1=df['Total Corpus (₹ Crs.)'].max(),
                  line=dict(color="orange", width=2, dash="dashdot"))
    
    # Add vertical line for zero corpus if applicable
    if not np.isnan(zero_corpus_age):
        fig.add_shape(type="line",
                      x0=zero_corpus_age, x1=zero_corpus_age,
                      y0=df['Total Corpus (₹ Crs.)'].min(), y1=df['Total Corpus (₹ Crs.)'].max(),
                      line=dict(color="red", width=2, dash="dashdot"))
    
    # Add stacked bar chart for positive and negative contributions
    df['Corpus Change Due to Returns'] = df['Returns on Corpus post-retirement'] / 10**7
    df['Corpus Change Due to Withdrawals'] = -df['Required Withdrawals'] / 10**7
    
    fig.add_trace(go.Bar(
        x=df['Age'],
        y=df['Corpus Change Due to Returns'],
        name='Returns on Corpus',
        marker_color='rgb(26, 118, 255)'
    ))
    
    fig.add_trace(go.Bar(
        x=df['Age'],
        y=df['Corpus Change Due to Withdrawals'],
        name='Withdrawals',
        marker_color='rgb(255, 0, 0)'
    ))
    
    fig.update_layout(
        title='Financial Independence Retire Early (FIRE) Plan',
        xaxis_title='Age',
        yaxis_title='Total Corpus (₹ Crs.)',
        barmode='relative',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        template='plotly_dark',
        hovermode='x unified',
        height=600
    )
    
    fig.show()
    
    # Display the table with formatted values
    formatted_df = df[['Age', 'Annual Salary (₹ Crs.)', 'Annual Investments (₹ Crs.)', 'Annual Expenses (pre-retirement) (₹ Crs.)', 'Future Value of Investments at Retirement (₹ Crs.)', 'Future Value of est. Expense (₹ Crs.)', 'Total Corpus (₹ Crs.)']].round(3)
    display(HTML(formatted_df.to_html(index=False)))

# Create interactive widgets
current_age_slider = widgets.IntSlider(value=34, min=25, max=60, step=1, description='Current Age', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
target_retirement_age_slider = widgets.IntSlider(value=55, min=50, max=70, step=1, description='Retirement Age', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
current_salary_slider = widgets.IntSlider(value=5000000, min=100000, max=20000000, step=100000, description='Current Salary (₹)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
salary_growth_rate_slider = widgets.FloatSlider(value=0.10, min=0.00, max=0.20, step=0.01, description='Salary Growth % (est. next 5 yrs.)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
percent_earnings_invested_slider = widgets.FloatSlider(value=0.30, min=0.05, max=0.50, step=0.01, description='% Earnings Invested', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
annual_returns_pre_retirement_slider = widgets.FloatSlider(value=0.12, min=0.05, max=0.20, step=0.01, description='Pre-Ret. ROI (on investment)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
annual_returns_post_retirement_slider = widgets.FloatSlider(value=0.06, min=0.02, max=0.10, step=0.01, description='Post-Ret. ROI (on corpus)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
initial_inflation_rate_slider = widgets.FloatSlider(value=0.07, min=0.02, max=0.10, step=0.01, description='Inflation % (est. next 5 yrs.)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
pre_retirement_expenses_slider = widgets.IntSlider(value=3178458, min=600000, max=3600000, step=10000, description='Pre-Ret. Expenses (₹)', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))

# Create VBox to hold all sliders
widget_box = VBox([
    current_age_slider,
    target_retirement_age_slider,
    current_salary_slider,
    salary_growth_rate_slider,
    percent_earnings_invested_slider,
    annual_returns_pre_retirement_slider,
    annual_returns_post_retirement_slider,
    initial_inflation_rate_slider,
    pre_retirement_expenses_slider
])

# Create an interactive output area
output = widgets.interactive_output(calculate_fire, {
    'current_age': current_age_slider,
    'target_retirement_age': target_retirement_age_slider,
    'current_salary': current_salary_slider,
    'salary_growth_rate': salary_growth_rate_slider,
    'percent_earnings_invested': percent_earnings_invested_slider,
    'annual_returns_pre_retirement': annual_returns_pre_retirement_slider,
    'annual_returns_post_retirement': annual_returns_post_retirement_slider,
    'initial_inflation_rate': initial_inflation_rate_slider,
    'pre_retirement_expenses': pre_retirement_expenses_slider
})

# Display the widgets and output
display(widget_box, output)

Output()